In [1]:
import os
import pandas as pd
from labels import LABELS
from setfit import SetFitModel, Trainer, TrainingArguments
from datasets import load_dataset

# dataset = load_dataset("ethos", "multilabel")

os.environ["TOKENIZERS_PARALLELISM"] = "false"

data = "./dataset_resume.csv"
df = pd.read_csv(data)

df_new = df.copy()
# on stocke la classification
features = df_new[LABELS]
# création d'une colonne labels pour représenter tous les labels de la description
df_new["labels"] = pd.Series().fillna("")  

/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from sklearn.model_selection import train_test_split

def setup(df):
    for index, row in df.iterrows():
        # s'il y a un résumé, on l'utilise comme description
        df.loc[index, "description"] = (
                row.resume
                if isinstance(row.resume, str)
                else row.description
            )
    
        features = row[LABELS]
        df.at[index, "labels"] = [feature for feature in features]
    
    df.drop(columns=['resume', 'title'], inplace=True)
    df.drop(columns=LABELS, inplace=True)
    # suppression de l'index
    df = df.iloc[:, 1:]

    train_df, test_data = train_test_split(
        df, test_size=0.2, random_state=42, shuffle=True
    )

    val_df, test_df = train_test_split(test_data, test_size=0.5)

    return train_df, test_df, val_df

train_df, test_df, val_df = setup(df_new)


In [7]:
from datasets import Dataset

train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
val_ds = Dataset.from_pandas(val_df)

#val_df.head(2)
test_ds.data

InMemoryTable
description: string
labels: list<item: double>
  child 0, item: double
__index_level_0__: int64
----
description: [[""Mon sujet, c’est la guerre, et ce qu'elle a de pitoyable." Wilfred Owen a écrit des textes poignants sur les ravages de la guerre… Le temps d'une randonnée, écoutez ses poèmes émouvants.  Gratuit. Inscription obligatoire sur le site du département en cliquant ici.","Venez participer au loto familial organisé par le collectif "Séjour Montagne 2023" ! Le Dimanche 10 Septembre à 14h ( ouverture des portes à 13h30) à la salle des fêtes de Busigny ! - 4 séries loto (1€ le carton) - 1 bingo (2 € le carton) - 1 loto chinois Paiement des cartons à l'inscription A gagner : 2 cartes cadeaux d'une valeur de 100€ et pleins d'autres lots de valeur ! Petite restauration sur place ( nourriture / alimentation extérieure interdite) Inscriptions obligatoires avant le 05 Septembre aux numéros suivant : Centre social La Passerelle : 03.27.76.26.75 EVS de Ligny : 07.84.32.15.3

In [8]:
model = SetFitModel.from_pretrained("dangvantuan/sentence-camembert-base", multi_target_strategy="one-vs-rest")

/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [9]:
args = TrainingArguments(
    batch_size=3,
    num_epochs=1,
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    column_mapping={"description": "text", "labels": "label"}
)


Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset
/Users/maximekuil/Documents/Simplon/fine-tune-classification/.venv/lib/python3.12/site-packages/datasets/utils/_dill.py:379: DeprecationWarning: co_lnotab is deprecated, use co_lines instead.
  obj.co_lnotab,  # for < python 3.10 [not counted in args]
Map: 100%|██████████| 312/312 [00:00<00:00, 14749.53 examples/s]


In [10]:
trainer.train()

***** Running training *****
  Num unique pairs = 74524
  Batch size = 3
  Num epochs = 1
  Total optimization steps = 24842
  0%|          | 0/24842 [00:00<?, ?it/s]

                                                 

{'embedding_loss': 0.411, 'learning_rate': 8.048289738430585e-09, 'epoch': 0.0}


KeyboardInterrupt: 